In [1]:
from random import Random
from uuid import uuid4

In [2]:
SEED = 5
random_gen = Random(x = SEED)
def generate_pnts(N = 1000):
  lst_x, lst_y = [], []
  for _ in range(N):
    lst_x.append(random_gen.uniform(a = 0, b = 1))
    lst_y.append(random_gen.uniform(a = 0, b = 1))
  return lst_x, lst_y
data_x, data_y = generate_pnts()

In [3]:
def calc_grad(x_p, y_p, batch_x, batch_y):
    sum_x, sum_y = 0., 0.
    for x_i, y_i in zip(batch_x, batch_y):
        inv_sqrt = ((x_i - x_p) ** 2 + (y_i - y_p) ** 2) ** (-0.5)
        sum_x += inv_sqrt * (x_i - x_p)
        sum_y += inv_sqrt * (y_i - y_p)
    return - sum_x / len(batch_x), - sum_y / len(batch_x)

In [9]:
x_p , y_p = 0.3, 0.3
calc_grad(x_p, y_p, data_x, data_y)

(-0.31721406583173956, -0.32132827163992495)

In [4]:
class CompNode:
        def __init__(self, val, children = [], op = "assign"):
            self.val = val
            self.children = children
            self.grad = 0
            self.op = op
            self.backward_prop = lambda : None
            self.identity = uuid4()

        def __to_comp_node(self, obj):
            if not isinstance(obj, CompNode):
                return CompNode(val = obj)
            else:
                return obj
        # you are given an example of doing a subtraction operation
        # and defining a method to do the subtraction backward propagation.
        def __sub__(self, other):
            other = self.__to_comp_node(other)
            out = CompNode(val = self.val - other.val,
                            children = [self, other],
                            op = "sub")
            def _backward_prop():
                self.grad += out.grad * 1
                other.grad += out.grad * (-1)
            out.backward_prop = _backward_prop
            return out
        def __rsub__(self, other):
            other = self.__to_comp_node(other)
            return other - self
        # Implement the add, multiplication, power functions
        def __add__(self, other):
            # write your code here
            other = self.__to_comp_node(other)
            out = CompNode(val = self.val + other.val,
                            children=[self, other],
                            op="add")
            def _backward_prop():
                # write your code here
                self.grad += out.grad * 1
                other.grad += out.grad * 1
            out.backward_prop = _backward_prop
            # make sure to return the output
            return out
        def __radd__(self, other):
            other = self.__to_comp_node(other)
            return other + self
        def __mul__(self, other):
            # write your code here
            other = self.__to_comp_node(other)
            out = CompNode(val=self.val * other.val,
                            children=[self, other],
                            op="mul")
            def _backward_prop():
                # write your code here
                self.grad += out.grad * other.val
                other.grad += out.grad * self.val
            out.backward_prop = _backward_prop
            # make sure to return the output
            return out
        def __rmul__(self, other):
            other = self.__to_comp_node(other)
            return other * self
        def __pow__(self, exponent):
            if not isinstance(exponent, (int, float)):
                raise ValueError("Unsupported types")
            # write your code here
            out = CompNode(val=self.val ** exponent,
                            children=[self],
                            op="pow")
            def _backward_prop():
                # write your code here
                self.grad += out.grad * exponent * (self.val ** (exponent - 1))
            # make sure to return the output
            out.backward_prop = _backward_prop
            return out
        def __eq__(self, other):
            return self.val == other.val
        def __repr__(self):
            return f"op: {self.op} | val: {self.val:.4f} | children: {len(self.children)} | grad: {self.grad}"
        def __hash__(self):
            return int(self.identity)
        def topo_sort (self, collect_edges = False):
            res = []
            visited = set()
            if collect_edges : edges = []
            def visit(node):
                if node not in visited:
                    visited.add(node)
                    for child in node.children:
                        if collect_edges:
                            edges.append((child, node))
                        visit(child)
                    res.append(node)
            visit(self)
            if collect_edges:
                return res, edges
            return res
        def backward(self):
            # implement backward function
            # to do back propagation
            # for all nodes
            nodes = self.topo_sort()
            self.grad = 1
            for node in reversed(nodes):
                node.backward_prop()

In [5]:
def loss_graph(x_p, y_p, data_x, data_y):
  loss = 0
  for curr_x, curr_y in zip(data_x, data_y):
    Ix, Iy = x_p - curr_x, y_p - curr_y
    g_x, g_y = Ix ** 2, Iy ** 2
    M = g_x + g_y
    loss+= M** 0.5
  return (1 / len(data_x)) * loss

In [10]:
x_p, y_p = CompNode(val=0.3), CompNode(val=0.3)
curr_loss = loss_graph(x_p, y_p, data_x, data_y)
curr_loss.backward()

In [11]:
x_p, y_p

(op: assign | val: 0.3000 | children: 0 | grad: -0.3172140658317395,
 op: assign | val: 0.3000 | children: 0 | grad: -0.32132827163992483)